In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
CSV_PATH = Path("log/classification/msg_cls4A_e40_bs8_pts4096_split40shot_4/logs/pred_per_sample_2024-08-11_12-42.csv")

df = pd.read_csv(CSV_PATH)
df.head()

,sample_path,target_name,target_id,pred_choice,bee,butterfly,dragonfly,wasp
0,butterfly/hn-but-2_2_50.csv,butterfly,1,0,-0.4096,-5.0402,-1.2896,-3.0095
1,butterfly/hn-but-2_42_28.csv,butterfly,1,2,-1.6580,-5.4889,-0.2559,-3.5032
2,butterfly/hn-but-2_42_39.csv,butterfly,1,2,-1.2191,-3.0213,-0.4741,-3.8297
3,butterfly/hn-but-2_42_44.csv,butterfly,1,2,-1.3440,-4.5612,-0.3677,-3.5072
4,butterfly/hn-but-2_37_1.csv,butterfly,1,1,-1.3625,-0.4970,-2.4507,-3.3195


In [4]:
def f2(v:str):
    vs = v.split("/")[-1].split(".")[-2].split("_")[-3:]
    return vs

df['scene'], df['instance'], df['frag'] = zip(*df['sample_path'].map(f2))
df.head()

,sample_path,target_name,target_id,pred_choice,bee,butterfly,dragonfly,wasp,scene,instance,frag
0,butterfly/hn-but-2_2_50.csv,butterfly,1,0,-0.4096,-5.0402,-1.2896,-3.0095,hn-but-2,2,50
1,butterfly/hn-but-2_42_28.csv,butterfly,1,2,-1.6580,-5.4889,-0.2559,-3.5032,hn-but-2,42,28
2,butterfly/hn-but-2_42_39.csv,butterfly,1,2,-1.2191,-3.0213,-0.4741,-3.8297,hn-but-2,42,39
3,butterfly/hn-but-2_42_44.csv,butterfly,1,2,-1.3440,-4.5612,-0.3677,-3.5072,hn-but-2,42,44
4,butterfly/hn-but-2_37_1.csv,butterfly,1,1,-1.3625,-0.4970,-2.4507,-3.3195,hn-but-2,37,1


In [27]:
aggs = {
    "target_name":"first",
    "target_id":"first",
    "bee":"mean",
    "butterfly":"mean",
    "dragonfly":"mean",
    "wasp":"mean",
    "pred_choice":"nunique",
    # "pred_choice":pd.Series.mode,
}

df1 = df.groupby(["scene","instance"]).agg(aggs).rename(columns={'pred_choice': 'nunique'})
df1[df1["target_name"]=="dragonfly"]

df2 = df1[["bee", "butterfly", "dragonfly", "wasp"]].idxmax(axis=1).rename("pred_name")
df1 = pd.concat([df1,df2], axis=1)

classes = ["bee", "butterfly", "dragonfly", "wasp"]
cm = {v:k for k,v in enumerate(classes)}
df1["pred_id"] = df1["pred_name"].map(cm)
display(df1)

instance_count = df1.index.__len__()
num_correct = (df1["target_name"] == df1["pred_name"]).sum()
acc = num_correct/instance_count
print(f"Overall Grouped-By-Instance Accuracy {acc:.3f}; Instances={instance_count}; Correct={num_correct}")


target_name  target_id       bee  butterfly  dragonfly  \
scene    instance                                                          
hn-bee-1 0                bee          0 -1.594000  -0.632800  -1.443100   
         11               bee          0 -0.375500  -3.821350  -1.711900   
         3                bee          0 -3.048386  -2.278786  -1.285629   
         4                bee          0 -1.918600  -2.054100  -0.377400   
         9                bee          0 -2.586100  -0.332500  -1.736900   
...                       ...        ...       ...        ...        ...   
mu-3     24               bee          0 -0.175692  -3.927100  -4.627892   
         31               bee          0 -0.903200  -4.835014  -2.256529   
         55               bee          0 -0.138061  -4.145754  -3.296775   
         6                bee          0 -0.767660  -3.981580  -2.484110   
         9                bee          0 -0.306430  -5.002006  -3.140839   

                       wasp  nunique  pred_name  pred_id  
scene    instance                                         
hn-bee-1 0        -3.534800        1  butterfly        1  
         11       -4.764450        1        bee        0  
         3        -4.913886        3  dragonfly        2  
         4        -3.276700        1  dragonfly        2  
         9        -4.037500        1  butterfly        1  
...                     ...      ...        ...      ...  
mu-3     24       -4.803438        2        bee        0  
         31       -4.349186        2        bee        0  
         55       -4.610064        1        bee        0  
         6        -3.949570        3        bee        0  
         9        -3.885232        4        bee        0  

[134 rows x 9 columns]

134
108
Grouped-By-Instance Accuracy 0.8059701492537313


In [33]:

unq = df1["target_name"].unique()
accs = []
for clas in unq:
    dfc = df1[df1["target_name"]==clas]
    instance_count = dfc.index.__len__()
    num_correct = (dfc["target_name"] == dfc["pred_name"]).sum()
    acc = num_correct/instance_count
    accs.append(acc)
    print(f"{clas:<10}: Grouped-By-Instance Accuracy {acc:.3f}; Instances={instance_count}; Correct={num_correct}")

print("Unweighted mean", np.mean(accs))
    

bee       : Grouped-By-Instance Accuracy 0.700; Samples=20; Correct=14
butterfly : Grouped-By-Instance Accuracy 0.312; Samples=16; Correct=5
dragonfly : Grouped-By-Instance Accuracy 0.946; Samples=93; Correct=88
wasp      : Grouped-By-Instance Accuracy 0.200; Samples=5; Correct=1
Unweighted mean 0.5396841397849462
